# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [45]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 로드
- 등록정보&제원정보(말소적용)
- 저감장치 부착이력

In [2]:
ods_fold = 'D:\과업\[한국환경공단]자동차_환경_빅데이터_구축_2차\data\ODS'

In [4]:
# about 16s
# 등록&제원 정보(말소적용)
df = pd.read_csv(os.path.join(ods_fold, '[ODS][G4]등록정보&제원정보(말소적용)(2022.12.29).csv'), low_memory=False)
# 저감장치 부착이력(중복 차대번호 처리)
dpf = pd.read_csv(os.path.join(ods_fold, '[ODS]저감장치부착이력(중복차대처리)(한글컬럼)(2022.12.29).csv'), low_memory=False, encoding='cp949')

In [23]:
# about 2s
# 저감장치 부착이력
raw_dpf = pd.read_csv(os.path.join(ods_fold, '[ODS]저감장치부착이력(한글컬럼)(2022.12.29).csv'), low_memory=False)

# [분석]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511251 entries, 0 to 1511250
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1511251 non-null  object 
 1   차량등록번호       1511251 non-null  object 
 2   본거지법정동코드     1511251 non-null  int64  
 3   소유자구분        1511251 non-null  object 
 4   차종_등록정보      1511249 non-null  object 
 5   용도           1511251 non-null  object 
 6   차대번호         1511251 non-null  object 
 7   최초등록일자       1511251 non-null  int64  
 8   차량연식         1511251 non-null  int64  
 9   차량제작일자       1342461 non-null  float64
 10  배출가스검사유효일자   1511239 non-null  float64
 11  제원관리번호       1511251 non-null  object 
 12  배출가스인증번호     1387542 non-null  object 
 13  배출가스등급       1511251 non-null  object 
 14  저공해조치종류      189 non-null      float64
 15  저공해조치구조변경YN  1511251 non-null  object 
 16  차량말소YN       1511251 non-null  object 
 17  취득일자         1418010 non-null  object 
 18  취득

In [6]:
df.shape

(1511251, 40)

In [7]:
dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809255 entries, 0 to 809254
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   저감장치종류      585410 non-null  object 
 1   TDMS부착여부    204903 non-null  object 
 2   구조변경일자      809255 non-null  int64  
 3   부착업체명       800321 non-null  object 
 4   부착일자        809254 non-null  float64
 5   탈거승인일       169 non-null     float64
 6   차량번호        809255 non-null  object 
 7   차대번호        809255 non-null  object 
 8   저감장치반납여부    324650 non-null  object 
 9   저감장치구분      809255 non-null  object 
 10  삼원촉매장치반납여부  13688 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 67.9+ MB


In [8]:
dpf.shape

(809255, 11)

In [24]:
raw_dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810777 entries, 0 to 810776
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   저감장치종류      586898 non-null  object 
 1   TDMS부착여부    204951 non-null  object 
 2   구조변경일자      810777 non-null  int64  
 3   부착업체명       801632 non-null  object 
 4   부착일자        810775 non-null  float64
 5   탈거승인일       178 non-null     float64
 6   차량번호        810777 non-null  object 
 7   차대번호        810777 non-null  object 
 8   저감장치반납여부    326030 non-null  object 
 9   저감장치구분      810777 non-null  object 
 10  삼원촉매장치반납여부  13694 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 68.0+ MB


In [25]:
raw_dpf.shape

(810777, 11)

## 차대번호 기준 병합

In [9]:
total = df.merge(dpf, on='차대번호', how='left')
total.shape

(1511251, 50)

In [11]:
total['구조변경일자'].isnull().sum()

1511215

In [39]:
raw_total = df.merge(raw_dpf, on='차대번호', how='left')
raw_total.shape

(1511251, 50)

## 저감 데이터에서 등록&제원 데이터와 매칭되지 않은 차대번호 샘플

In [33]:
miss_match_vin = list(set(dpf['차대번호']) - set(df['차대번호']))
len(miss_match_vin)

809219

In [34]:
match_vin = list(set(dpf['차대번호']) & set(df['차대번호']))
len(match_vin)

36

In [35]:
match_vin

['KMFGA17HP8C076574',
 'KMFGA17HP8C075775',
 'KMHSJ81WP7U231639',
 'KMHSJ81WP7U204822',
 'KMFLA18KP8C028113',
 'KL5UY61YD7P000752',
 'KL3K9ARF57K000004',
 'KMFGA17HP8C081531',
 'KMHSJ81VP8U235201',
 'KMHSJ81WP7U220194',
 'KMHSJ81VP8U267989',
 'KL5UY61YD7P000720',
 'KMHSH81WP7U191739',
 'KMHSH81WP7U251766',
 'KNCSE01428K330851',
 'KMHSJ81WP7U200814',
 'KPACA4AB18P039175',
 'KMJHD17HP8C000170',
 'KMFGA17HP8C098864',
 'KNCSE03428K344383',
 'KMFGA17HP8C076943',
 'KMFLA18KP8C028034',
 'KNCSJZ74AAK432404',
 'KMHSJ81WP7U186356',
 'KMFGA17HP8C076295',
 'KMHSH81VP8U267598',
 'KMFGA17HP8C094884',
 'KMJHD17PP7C036961',
 'KL5UY61YD7P000894',
 'KL3K9ARF18K000823',
 'KL5UY61YD7P000803',
 'KPACA4AB18P035816',
 'KMFGA17HP8C086533',
 'KMJHD17PP7C034127',
 'KMFGA17HP8C075927',
 'KL3C6CHF18K003487']

In [36]:
match_dpf = pd.DataFrame()
for one in tqdm(match_vin):
    temp = raw_dpf[raw_dpf['차대번호'] == one]
    match_dpf = pd.concat([match_dpf, temp], ignore_index=True)
match_dpf.shape

100%|██████████| 36/36 [00:01<00:00, 22.36it/s]


(36, 11)

In [37]:
match_dpf.sort_values(['차대번호', '구조변경일자', '부착일자'])

,저감장치종류,TDMS부착여부,구조변경일자,부착업체명,부착일자,탈거승인일,차량번호,차대번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부
35,복합대형,N,20191216,㈜주왕,20191216.0,NaN,충남80아2361,KL3C6CHF18K003487,NaN,1종,NaN
29,자연대형,N,20191011,(주)명성테크,20191011.0,NaN,부산91아2052,KL3K9ARF18K000823,NaN,1종,NaN
6,자연대형+SCR,N,20190927,(주)서천제일자동차공업사,20190927.0,NaN,서울87아7110,KL3K9ARF57K000004,NaN,1종+SCR,NaN
11,복합대형,N,20160126,북항자동차정비검사소,20160126.0,NaN,서울74바1057,KL5UY61YD7P000720,NaN,1종,NaN
5,자연대형+SCR,N,20160826,형제자동차공업사(부천),20160826.0,NaN,인천73바9377,KL5UY61YD7P000752,NaN,1종+SCR,NaN
30,복합대형,N,20160127,북항자동차정비검사소,20160127.0,NaN,서울72바8684,KL5UY61YD7P000803,NaN,1종,NaN
28,복합대형,N,20160310,한라자동차정비㈜,20160310.0,NaN,71구1475,KL5UY61YD7P000894,NaN,1종,NaN
1,복합중형,NaN,20111126,삼정자동차정비㈜,20111126.0,NaN,인천82바1830,KMFGA17HP8C075775,NaN,1종,NaN
34,복합중형,NaN,20101022,㈜영남중공업,20101022.0,NaN,서울81사6991,KMFGA17HP8C075927,NaN,1종,NaN
24,복합중형,N,20181114,(주)현대자동차블루핸즈서인천서비스,20181114.0,NaN,인천83바4554,KMFGA17HP8C076295,NaN,1종,NaN


In [41]:
match_dpf.shape

(36, 11)

In [38]:
match_dpf.duplicated('차대번호').sum()

0

### [출력] 매칭되는 차대번호 샘플

In [40]:
match_dpf.sort_values(['차대번호', '구조변경일자', '부착일자']).to_excel('analysis2/[BD3][G4][통계]저감장치부착이력_등록&제원정보_매칭_차대번호_샘플(말소적용).xlsx', index=False)

## 분류 통계
- 중복 차대번호 제거
    - 최초등록일자, 배출가스검사유효일자 : 최신순(ascending=False)
- 분류
    - 연료 : 휘발유가스, 경유
    - 차량제작일자
    - 구조변경일자 여부

In [42]:
total.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자', '제원관리번호', '배출가스인증번호',
       '배출가스등급', '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명',
       '차종_제원정보', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자',
       '구동형식', '변속기종류', '저감장치종류', 'TDMS부착여부', '구조변경일자', '부착업체명', '부착일자',
       '탈거승인일', '차량번호', '저감장치반납여부', '저감장치구분', '삼원촉매장치반납여부'],
      dtype='object')

### 중복 차대번호 처리
- '최초등록일자', '배출가스검사유효일자', ascending=False

In [47]:
total.shape[0], len(total['차대번호'].unique())

(1511251, 1511211)

In [49]:
total['dupl'] = total['차대번호'].duplicated().to_list()
total.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,TDMS부착여부,구조변경일자,부착업체명,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl
0,CN-62-T-2213-01,서울2흐1820,1159010200,주민,승용,자가용,CN-62-T-2213,19650209,2002,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,SL68T032460-01,서울5가1334,1165010100,기타,승용,자가용,SL68T032460,19750127,1968,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2T1CF2897YC359923-01,16다1728,1117012400,기타,승용,자가용,2T1CF2897YC359923,19821231,2000,19821231.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,KNADA2322HS004198-01,전북1나2309,4574033530,주민,승용,자가용,KNADA2322HS004198,19870411,1987,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,KNADA2322HS004932-01,경기30다4720,4159025323,주민,승용,자가용,KNADA2322HS004932,19870421,1987,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [68]:
dupl_vin = total[total['dupl'] == True].sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=False).reset_index(drop=True)
dupl_vin.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,TDMS부착여부,구조변경일자,부착업체명,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl
0,1GBEL19W3XB192382-02,39고6855,1141011800,주민,승용,자가용,1GBEL19W3XB192382,20091013,1999,19991231.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,KMHBF31FPMV273591-03,57구7651,4513040021,기타,승용,자가용,KMHBF31FPMV273591,20040714,1991,19910101.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,KMHBF21DPNU459285-04,57구3409,4513040021,기타,승용,자가용,KMHBF21DPNU459285,20040302,1992,19920101.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,KMHBF31FPNU503892-03,16다1105,1117012400,기타,승용,자가용,KMHBF31FPNU503892,20040112,1992,19921231.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,KLAJA19Y1SB055388-03,대구30고9995,2720010300,기타,승용,자가용,KLAJA19Y1SB055388,20031204,1995,19950623.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


#### [출력] 중복 차대번호 샘플

In [69]:
dupl_vin.to_excel('analysis2/[BD3][G4][샘플]등록&제원-저감_중복차대번호(말소적용).xlsx')

In [70]:
total1 = total.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
total1.shape

(1511211, 51)

### [Case 5.]차량연료별 분류
- 차량 대수
    - 휘발류•가스('휘발유', '알코올', 'LGP(액화석유가스)', 'CNG(압축천연가스)')
    - 경유

In [ ]:
total1['연료'].unique()

array(['경유', '휘발유', nan, '기타연료', 'LGP(액화석유가스)', 'CNG(압축천연가스)', '알코올'],
      dtype=object)

In [71]:
total1[ (total1['연료'] == '휘발유') | (total1['연료'] == '알코올') | (total1['연료'] == 'LGP(액화석유가스)') | (total1['연료'] == 'CNG(압축천연가스)') ].shape

(358404, 51)

In [72]:
total1[total1['연료'] == '경유'].shape

(1152697, 51)

In [73]:
num_by_fuel = pd.DataFrame({'연료':['휘발유_가스', '경유'], 
                            '수(차대번호)':[
                                total1[ (total1['연료'] == '휘발유') | (total1['연료'] == '알코올') | (total1['연료'] == 'LGP(액화석유가스)') | (total1['연료'] == 'CNG(압축천연가스)') ].shape[0], 
                                total1[total1['연료'] == '경유'].shape[0]
                                ]
                            })
num_by_fuel

,연료,수(차대번호)
0,휘발유_가스,358404
1,경유,1152697


In [86]:
fuel_list = []
for one in tqdm(total1['연료']):
    if one == '휘발유' or one == '알코올' or one == 'LGP(액화석유가스)' or one == 'CNG(압축천연가스)':
        fuel_list.append('휘발유_가스')
    elif one == '경유':
        fuel_list.append('경유')
    else:
        fuel_list.append('기타')
len(fuel_list)

100%|██████████| 1511211/1511211 [00:00<00:00, 1634310.81it/s]


1511211

In [87]:
total1['fuel'] = fuel_list

### [Case 5.]차량제작일자별 분류
- 휘발류•가스('휘발유', 'LGP(액화석유가스)', '알코올', 'CNG(압축천연가스)')
    - A : 1988.01.01 ~ 2000.12.31
    - B : ~ 1997.12.31
- 경유(DPF : 구조변경일자 여부)
    - A : DPF, 2008.01.01 ~
    - B : DPF, ~ 2007.12.31
    - C : 2008.01.01 ~
    - D : ~ 2007.12.31

In [74]:
total1['차량제작일자'].describe()

count    1342438.00
mean    20072068.99
std        52081.54
min      2100819.00
25%     20070816.00
50%     20090306.00
75%     20100619.00
max     20190926.00
Name: 차량제작일자, dtype: float64

In [76]:
total1['구조변경일자'].describe()

count         36.00
mean    20168754.36
std        33314.56
min     20101022.00
25%     20157721.50
50%     20190755.50
75%     20191120.25
max     20200211.00
Name: 구조변경일자, dtype: float64

In [77]:
total1['구조변경일자'].isnull().sum()

1511175

In [78]:
total1['저감장치종류'].unique()

array([nan, '복합소형', '복합중형', '중형', '자연대형', '복합대형', '자연대형+SCR'],
      dtype=object)

In [79]:
total1['저감장치종류'].isnull().sum()

1511175

In [80]:
str(np.nan) == 'nan'

True

In [93]:
grade_list = []
for f, y, d in tqdm(zip(total1['연료'], total1['차량제작일자'], total1['구조변경일자'])):
    if ( (f == '휘발유') or (f == 'LGP(액화석유가스)') or (f == '알코올') or (f == 'CNG(압축천연가스)') ) and (19980101 <= y <= 20001231):
        grade_list.append('A')
    elif (f == '휘발유' or f == 'LGP(액화석유가스)' or f == '알코올' or f == 'CNG(압축천연가스)') and (y <= 19971231):
        grade_list.append('B')
    elif (f == '경유') and (y >= 20080101) and (str(d) != 'nan'):
        grade_list.append('A')
    elif (f == '경유') and (y <= 20071231) and (str(d) != 'nan'):
        grade_list.append('B')
    elif (f == '경유') and (y >= 20080101) and (str(d) == 'nan'):
        grade_list.append('C')
    elif (f == '경유') and (y <= 20071231) and (str(d) == 'nan'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

1511211it [00:02, 577705.33it/s]


1511211

In [94]:
total1['Grade'] = grade_list

In [100]:
total1.groupby(['fuel'])['차대번호'].count()

fuel
경유        1152697
기타            110
휘발유_가스     358404
Name: 차대번호, dtype: int64

In [95]:
total1.groupby(['fuel', 'Grade'])['차대번호'].count()

fuel    Grade
경유      A            10
        B            26
        C        934443
        D        218215
        X             3
기타      X           110
휘발유_가스  A         43185
        B        128204
        X        187015
Name: 차대번호, dtype: int64

In [99]:
total1[(total1['fuel'] == '휘발유_가스') & (total1['Grade'] == 'X')]

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,부착업체명,부착일자,탈거승인일,차량번호,저감장치반납여부,저감장치구분,삼원촉매장치반납여부,dupl,fuel,Grade
55,1C3EW65G51V703776-02,14저9873,2729011100,주민,승용,자가용,1C3EW65G51V703776,20200421,2001,20190923.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
56,1C3EW65G51V703776-01,14저9733,4113111000,주민,승용,자가용,1C3EW65G51V703826,20200410,2001,20011231.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
106,KL90F4ABGNZ100001-01,01고3520,4146310700,법인,승용,자가용,KL90F4ABGNZ100001,20190621,1992,20190530.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
122,KMHEP41BP2A644387-04,04보2050,4122025041,기타,승용,자가용,KMHEP41BP2A644387,20190422,2002,20020312.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
155,KMHEP41BP1A451538-02,45라6813,4785025000,기타,승용,자가용,KMHEP41BP1A451538,20190312,2001,20010305.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511205,438711300719충-01,서울2투6737,1168010500,사업자,승용,자가용,438711300719충,19870316,2007,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
1511206,KNAD2322JS055503-01,경기2머9331,4155025029,주민,승용,자가용,KNAD2322JS055503,19870101,1995,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
1511207,KNADA2322HS006936-01,경기2가9056,5011013700,주민,승용,자가용,KNADA2322HS006936,19860513,1987,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X
1511209,SL68T032460-01,서울5가1334,1165010100,기타,승용,자가용,SL68T032460,19750127,1968,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,휘발유_가스,X


#### [출력]

In [102]:
ex = pd.ExcelWriter('analysis2/[BD3][G4][통계]Case5_등급분류(중복차대제거)(말소적용).xlsx')
total1.groupby(['fuel'])['차대번호'].count().to_excel(ex, sheet_name='연료별')
total1.groupby(['fuel', 'Grade'])['차대번호'].count().to_excel(ex, sheet_name='등급별')
ex.save()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_22420\3080637872.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  ex.save()


# 코드 마지막